In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

Defining Resnet-34

In [10]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self,filters,strides=1,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides = strides, padding = 'same', use_bias = False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides = 1, padding = 'same', use_bias = False),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides = strides, padding = 'same', use_bias = False),
                keras.layers.BatchNormalization()]
            
    def call(self,inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
            
        return self.activation(Z + skip_Z)

In [12]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64,7,strides=2,input_shape = [224,224,3],padding='same',use_bias = False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation('relu'))
model.add(keras.layers.MaxPool2D(pool_size = 3, strides = 2, padding = 'same'))

prev_filter = 64
for filters in [64]*3 + [128]*4 + [256]*6 + [512]*3:
    strides = 1 if filters == prev_filter else 2
    model.add(ResidualUnit(filters,strides = strides))
    prev_filters = filters
    
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10,activation = 'softmax'))

## Working with Pre-trained Models

In [8]:
from sklearn.datasets import load_sample_image

import numpy as np

import tensorflow as tf
from tensorflow import keras

In [9]:
china = load_sample_image('china.jpg')/255
flower = load_sample_image('flower.jpg')/255

images = np.array([china,flower])

batch_size , height, width, channels = images.shape

In [12]:
model = keras.applications.resnet50.ResNet50(weights = 'imagenet')

102981632/102967424 [==============================] - 2s 0us/step


In [13]:
images_resized = tf.image.resize(images, [224, 224])

In [14]:
inputs = keras.applications.resnet50.preprocess_input(images_resized*255)

Y_probas = model.predict(inputs)
top_K = keras.applications.resnet50.decode_predictions(Y_probas,top=3)
for image_index in range(len(images)):
    print('Image #{}'.format(image_index))
    for class_id,name,y_proba in top_K[image_index]:
        print(' {} - {:12s} {:.2f}'.format(class_id,name,y_proba*100))
    print()

49152/35363 [=========================================] - 0s 1us/step
Image #0
 n02825657 - bell_cote    82.35
 n03877845 - palace       6.75
 n03781244 - monastery    4.19

Image #1
 n03530642 - honeycomb    49.81
 n13040303 - stinkhorn    33.96
 n02206856 - bee          4.35



## Transfer Learning 

In [2]:
import tensorflow_datasets as tfds

In [3]:
dataset,info = tfds.load('tf_flowers', as_supervised = True, with_info = True)

dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes

test_set_raw,valid_set_raw,train_set_raw = tfds.load('tf_flowers',split = ['train[:10%]', 'train[10%:25%]', 'train[25%:]'], as_supervised = True)

In [4]:
def preprocess(image, label):
    resized_image = tf.image.resize(image,[224,224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    
    return final_image, label

In [5]:
batch_size = 32
train_set_raw = train_set_raw.shuffle(1000)
train_set = train_set_raw.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)

In [6]:
base_model = keras.applications.xception.Xception(weights = 'imagenet',include_top = False) #don't load the top, since we will train ourselves

avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes,activation = 'softmax')(avg)
model = keras.Model(inputs = base_model.input , outputs = output)

for layer in base_model.layers: #freeze early layers in base model
    layer.trainable = False

In [7]:
early_stop = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights=True)

optimizer = keras.optimizers.SGD(learning_rate = 0.2, momentum = 0.9, decay = 0.01)
model.compile(loss='sparse_categorical_crossentropy',optimizer = optimizer, metrics = ['accuracy'])
history = model.fit(train_set,epochs =5, validation_data = valid_set)

Epoch 1/5
